<a href="https://colab.research.google.com/github/reginaofori/-Domain-Knowledge-into-PLMs-for-EBM/blob/main/Appendix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Haversine Function

In [1]:
import math

def haversine_distance(coords1, coords2):
    """
    This function calculates the Haversine distance between two points on the earth specified by latitude/longitude.

    Parameters:
    coords1 (dict): A dictionary containing 'latitude' and 'longitude' of the first location.
    coords2 (dict): A dictionary containing 'latitude' and 'longitude' of the second location.

    Returns:
    float: The Haversine distance between the two locations in kilometers.
    """

    # Convert degrees to radians
    RADIANS_PER_DEGREE = math.pi / 180

    # Convert latitudes and longitudes to radians
    latitude1 = coords1['latitude'] * RADIANS_PER_DEGREE
    longitude1 = coords1['longitude'] * RADIANS_PER_DEGREE
    latitude2 = coords2['latitude'] * RADIANS_PER_DEGREE
    longitude2 = coords2['longitude'] * RADIANS_PER_DEGREE

    # Calculate differences in coordinates
    d_lat = latitude2 - latitude1
    d_lon = longitude2 - longitude1

    # Haversine formula to calculate distance between two points on a sphere
    a = math.sin(d_lat / 2) ** 2 + math.cos(latitude1) * math.cos(latitude2) * math.sin(d_lon / 2) ** 2
    distance = 6371.01 * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return distance

### Net Present Value  Function

In [ ]:
def net_present_value(irr, years, initial_investment, average_yearly_revenue, recurrent_yearly_cost):
    """
    This function calculates the Net Present Value (NPV) of a series of cash flows.

    Parameters:
    irr (float): The discount rate or interest rate.
    years (int): The number of years for the cash flows.
    initial_investment (float): The initial investment amount.
    average_yearly_revenue (float): The average yearly revenue.
    recurrent_yearly_cost (float): The recurrent yearly cost.

    Returns:
    float: The calculated NPV.
    """

    # Initialize NPV with the negative initial investment
    npv = -initial_investment

    # Calculate NPV for each year
    for i in range(1, years + 1):
        # Calculate the net cash flow for the year
        cash_flow = average_yearly_revenue - recurrent_yearly_cost

        # Discount the cash flow and add it to the NPV
        npv += cash_flow / pow(1 + irr, i)

    return npv

### Get Closeby/Cluster Facilities Function

In [ ]:
def get_cluster_facilities(terminal_arr, facility_coords):
    """
    This function calculates the haversine distance from a facility to each terminal in a list,
    filters out terminals that are more than 150km away, sorts the remaining terminals by distance,
    and returns the five closest terminals.

    Parameters:
    terminal_arr (list): A list of dictionaries, each containing 'coords' key for a terminal.
    facility_coords (dict): A dictionary containing 'coords' key for the facility.

    Returns:
    list: A list of dictionaries for the five closest terminals within 150km, each dictionary contains 'coords' and 'distance'.
    """

    # Calculate the haversine distance from the facility to each terminal
    distances = []
    for terminal in terminal_arr:
        distance = haversine_distance(facility_coords['coords'], terminal['coords'])
        terminal['distance'] = distance
        distances.append(terminal)

    # Filter out terminals that are more than 150km away
    filtered_distances = [terminal for terminal in distances if 0 < terminal['distance'] <= 150]

    # Sort the terminals by distance, closest first
    sorted_distances = sorted(filtered_distances, key=lambda terminal: terminal['distance'])

    # Return the five closest terminals
    return sorted_distances[:5]

### Get Development Stratergy per Facilty Function

In [ ]:

def get_development_type(terrain, oil_volume, is_cluster, crude_to_facility, process_to_facility, process_to_terminal):
    """
    This function determines the development type based on various parameters.

    Parameters:
    terrain (str): The type of terrain (land, water, swarm).
    oil_volume (float): The volume of oil.
    is_cluster (bool): Whether it's a cluster.
    crude_to_facility (bool): Whether crude is to be transported to the facility.
    process_to_facility (bool): Whether processed oil is to be transported to the facility.
    process_to_terminal (bool): Whether processed oil is to be transported to the terminal.

    Returns:
    list: A list containing the development type and an array of development types.
    """

    # Initialize variables
    dev_type = ""
    dev_arr = []

    # Define minimum volumes for different terrains
    cluster_min_volume_land = 5  # Minimum acceptable Rerservoir volume for Land
    cluster_min_volume_water = 20  # Minimum acceptable Rerservoir volume for Water
    cluster_min_volume_swarm = 10  # Minimum acceptable Rerservoir volume for Swarm

    # Determine development type based on terrain and other parameters
    if terrain == "land":
        if 0 < oil_volume < cluster_min_volume_land:
            dev_type = "AL4" if is_cluster else "AL5"
        else:
            if crude_to_facility:
                dev_type = "AL1"
            if process_to_facility:
                dev_type = "AL2"
            if process_to_terminal:
                dev_type = "AL3"
        dev_arr.append(dev_type)

    elif terrain == "water":
        if 0 < oil_volume < cluster_min_volume_water:
            dev_type = "AW4" if is_cluster else "AW5"
        else:
            if crude_to_facility:
                dev_type = "AW1"
            if process_to_terminal:
                dev_type = "AW3"
        dev_arr.append(dev_type)

    elif terrain == "swarm":
        if 0 < oil_volume < cluster_min_volume_swarm:
            dev_type = "AS4" if is_cluster else "AS5"
        else:
            if crude_to_facility:
                dev_type = "AS1"
            if process_to_facility:
                dev_type = "AS2"
            if process_to_terminal:
                dev_type = "AS3"
        dev_arr.append(dev_type)

    return [dev_type, dev_arr]

### Completion Cost Function

In [ ]:

def completion_cost(terrain, new_wells, enterable_wells):
    """
    This function calculates the completion cost based on the terrain and the number of new and enterable wells.

    Parameters:
    terrain (str): The type of terrain (land, water, swarm).
    new_wells (str): The number of new wells as a string.
    enterable_wells (str): The number of enterable wells as a string.

    Returns:
    float: The calculated completion cost.
    """
    cost_land = 5  # Completion cost onshore
    cost_water = 20  # Completion cost offshore
    cost_swarm = 10  # Completion cost swarm

    # Convert the number of wells from string to integer
    new_wells = int(new_wells)
    enterable_wells = int(enterable_wells)

    # Calculate the completion cost based on the terrain
    if terrain.lower() == "land":
        return new_wells * cost_land + enterable_wells * 0.5 * cost_land
    elif terrain.lower() == "water":
        return new_wells * cost_water + enterable_wells * 0.5 * cost_water
    elif terrain.lower() == "swarm":
        return new_wells * cost_swarm + enterable_wells * 0.5 * cost_swarm

####

+----------------------------------+
|         Research Strategy        |
|                                  |
| +----------------------------+  |
| |    Case Study Approach     |  |
| |                            |  |
| | +---------+  +-----------+ |  |
| | | Phase 1 |  |  Phase 2   | |  |
| | |         |  |           | |  |
| | |Lit. Rev.|  |  Survey    | |  |
| | | & Expert|  | of Oil     | |  |
| | |Interviews|  | Field Ops.| |  |
| | +---------+  +-----------+ |  |
| +----------------------------+  |
+----------------------------------+
                |
                v
+----------------------------------+
|    Data Collection Methods       |
|                                  |
| +----------+  +---------+  +----+|
| | Literature|  | Expert  |  |Survey|
| | Review    |  |Interviews|  |     |
| +----------+  +---------+  +----+|
+----------------------------------+
                |
                v
+----------------------------------+
| Data Preparation and Formulation |
|                                  |
| +-------------------------------+|
| | Data Extraction & JSON Format  |
| | Function Development (Harvesine,|
| | NPV, Well Completion Cost)     |
| +-------------------------------+|
+----------------------------------+
                |
                v
+----------------------------------+
|         Data Analysis            |
|                                  |
| +-------------+  +-------------+ |
| | Development |  |  AHP Model  | |
| | Strategy    |  |             | |
| | Selection & |  |             | |
| | NPV Analysis|  |             | |
| +-------------+  +-------------+ |
|                                  |
|        Combined Analysis         |
+----------------------------------+
